# 指数相关性

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月8日

----------

- 指数间及指数与大宗商品间相关性分析；
- 指数：沪深规模指数、行业指数、主题指数，恒生指数、恒生国企指数、标普500指数；
- 大宗商品：黄金、白银、原油。


In [ ]:
import pandas as pd
import requests, json, os, re, time
import matplotlib.pyplot as plt
from datetime import timedelta,date,datetime

plt.rcParams['axes.unicode_minus']=False #用来正常显示负号  


#引入自定义库
import sys
sys.path.append('/home/jquser')

from pf_idx import Index
index=Index('csv','../Data/')

from pf_cmm import Cmm
cmm=Cmm('csv','../Data/')

数据引擎：运行于研究
指数框架：运行于研究
商品框架：运行于研究


## 1.数据准备

In [ ]:
#大宗商品
commodity_list=cmm.pool.track

#A股指数
index_list=index.pool.watch.keys()
#index.pool.qsc.keys()[0:2]+index.pool.kj.keys()+index.pool.zj.keys()+index.pool.hk.keys()+['SPX','DJIA','NDAQ','GDAXI']


#获取指数价格
def get_index_price(years=10):
    df=pd.DataFrame()
    for code in index_list:
        name=index.pool.track[code]
        price_df=index.data.read(code,items=['close'],years=years)
        price_df=price_df.rename(columns={'close':name})
        df=pd.concat([df,price_df],axis=1)
    return df


#获取大宗商品价格
def get_commodity_price(years=10):
    df=pd.DataFrame()
    for code in commodity_list:
        name=cmm.pool.track[code]
        price_df=cmm.data.read(code,items=['close'],years=years)
        price_df=price_df.rename(columns={'close':name})
        df=pd.concat([df,price_df],axis=1)
    return df


#整理数据
def get_all_price(years=10):
    index_df=get_index_price(years=years)
    commodity_df=get_commodity_price(years=years)
    df=pd.concat([index_df,commodity_df],axis=1)
    print '\r数据获取完成！',
    return df.dropna()

price_df=get_all_price()
price_df

## 1.计算相关性

In [6]:
#设置pd显示小数点位
pd.set_option('precision',4)
pd.option_context('display.float_format', lambda x: '%.4f' % x)
#设置pd显示行列数目
pd.set_option('display.max_rows',50)
pd.set_option('display.max_columns',50)
#计算相关性
corr_df=price_df.corr()
#保存结果
index.data.save('idx_relevance',corr_df)
corr_df 

,中证1000,全指消费,中证消费,沪深300,中证500,创业板指,标普500,全指信息,中证红利,恒生指数,全指医药,中证传媒,深证100,上证180,深证综指,全指证券,中证养老,食品饮料,中证环保,中证军工,全市场,恒生国企,全指金融,上证50,中小板指,布伦特原油,伦敦金,伦敦银
中证1000,1.000,-1.397e-01,-0.450,0.443,0.942,0.959,-0.740,0.960,0.348,-0.407,0.722,0.957,0.559,0.394,0.965,9.327e-01,0.863,-0.519,0.961,0.946,0.917,0.009,0.519,0.052,0.835,-0.518,-0.366,0.231
全指消费,-0.140,1.000e+00,0.941,0.743,0.126,-0.109,0.709,0.044,0.806,0.907,0.465,-0.220,0.677,0.746,0.107,3.111e-04,0.274,0.910,0.061,-0.176,0.233,0.861,0.632,0.879,0.321,0.723,0.403,0.103
中证消费,-0.450,9.411e-01,1.000,0.538,-0.187,-0.403,0.876,-0.266,0.616,0.959,0.195,-0.503,0.450,0.552,-0.214,-3.110e-01,-0.018,0.992,-0.250,-0.470,-0.084,0.780,0.408,0.784,0.040,0.814,0.483,0.005
沪深300,0.443,7.432e-01,0.538,1.000,0.695,0.486,0.171,0.609,0.964,0.597,0.811,0.406,0.966,0.993,0.633,5.960e-01,0.778,0.449,0.642,0.434,0.760,0.860,0.947,0.908,0.823,0.326,0.013,0.029
中证500,0.942,1.262e-01,-0.187,0.695,1.000,0.936,-0.539,0.972,0.599,-0.122,0.861,0.913,0.766,0.657,0.971,9.415e-01,0.962,-0.276,0.990,0.928,0.986,0.311,0.711,0.343,0.955,-0.290,-0.345,0.136
创业板指,0.959,-1.092e-01,-0.403,0.486,0.936,1.000,-0.710,0.981,0.345,-0.361,0.765,0.987,0.581,0.445,0.920,9.379e-01,0.903,-0.481,0.933,0.963,0.895,0.078,0.529,0.104,0.868,-0.432,-0.434,0.069
标普500,-0.740,7.094e-01,0.876,0.171,-0.539,-0.710,1.000,-0.608,0.293,0.849,-0.171,-0.784,0.036,0.213,-0.558,-6.122e-01,-0.409,0.911,-0.595,-0.735,-0.451,0.549,0.054,0.527,-0.364,0.848,0.462,-0.082
全指信息,0.960,4.356e-02,-0.266,0.609,0.972,0.981,-0.608,1.000,0.490,-0.224,0.832,0.952,0.701,0.566,0.964,9.535e-01,0.945,-0.350,0.967,0.939,0.954,0.204,0.646,0.244,0.928,-0.348,-0.338,0.138
中证红利,0.348,8.060e-01,0.616,0.964,0.599,0.345,0.293,0.490,1.000,0.672,0.727,0.258,0.917,0.961,0.560,4.957e-01,0.665,0.538,0.551,0.307,0.685,0.867,0.923,0.922,0.712,0.359,0.131,0.105
恒生指数,-0.407,9.072e-01,0.959,0.597,-0.122,-0.361,0.849,-0.224,0.672,1.000,0.231,-0.453,0.487,0.619,-0.171,-2.425e-01,0.032,0.942,-0.185,-0.412,-0.025,0.871,0.471,0.834,0.090,0.801,0.461,0.011


## 1.相关性图

In [ ]:
def heat(df):
    #相关性热力图
    #设置画板
    fig,ax=plt.subplots(figsize=(20,16),dpi=300) 
    #画图
    p=ax.imshow(df,cmap=plt.cm.jet,interpolation='none')
    #色板
    cbar=fig.colorbar(p,ax=ax)
    cbar.set_label('<　　　　　　弱　　　　　　相关性　　　　　　强　　　　　　>',fontsize=14)  
    #设置x轴刻度在上方
    # ax.xaxis.tick_top()
    #设置x轴刻度及文字
    ticks=range(len(df))
    labels=corr_df.columns  
    ax.invert_yaxis()
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels,fontsize=14,rotation=270)
    ax.set_yticks(ticks)
    ax.set_yticklabels(labels,fontsize=14)
    #标题
    ax.set_ylabel('<　　　　　　指数　　　　　　　　　　大宗商品　　　　　　>',fontsize=14)  
    ax.set_xlabel('<　　　　　　指数　　　　　　　　　　大宗商品　　　　　　>',fontsize=14) 
    plt.title('指数及大宗商品相关性 - 强弱图',fontsize=16)
    #强弱值
    for i in ticks:
        for j in ticks:
            ax.text(i-0.35,j-0.1,'%.2f'%(df.iloc[i,j]),fontsize=12,color='white')
    return fig
    #保存图片
    fig.savefig('index_relevance.png',dpi=120,bbox_inches='tight')

    
fig=heat(corr_df)    
index.value.chart.to_image(fig,'idx_relevance')